In [ ]:
#Import Dependencies
import os
import pandas as pd

In [ ]:
#Define pathways for datafiles
path_schools = os.path.join("Resources", "schools_complete.csv")
path_students = os.path.join("Resources", "students_complete.csv")

#Load in datafiles
schools_df = pd.read_csv(path_schools)
students_df = pd.read_csv(path_students)